<a href="https://colab.research.google.com/github/taegunYoun/AI-Powered-Automated-Blog/blob/main/AI_Powered_Automated_Blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# 크롤링할 값을 변수로 설정
query = "OpenAI"  # 검색어
start_date = "2025.02.01"  # 시작 날짜
end_date = "2025.03.31"  # 종료 날짜
max_news = 20  # 최대 뉴스 개수
news_per_page = 10  # 네이버 뉴스 검색 결과 한 페이지당 뉴스 개수

# 뉴스 데이터 저장용 리스트
data = []

# 여러 페이지에서 크롤링하기
for start in range(1, max_news, news_per_page):
    # URL 생성 (페이지네이션 반영)
    url = f"https://search.naver.com/search.naver?where=news&query={query}&ds={start_date}&de={end_date}&sort=0&start={start}"

    # 요청 보내기
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 뉴스 리스트 추출
    news_list = soup.select("ul.list_news > li.bx")

    # 뉴스 데이터 저장
    for news in news_list:
        # 기사 제목과 URL
        title_tag = news.select_one("a.news_tit")
        if not title_tag:
            continue  # 제목이 없으면 스킵
        title = title_tag.get_text()
        article_url = title_tag['href']

        # 기사 요약 내용
        summary_tag = news.select_one("a.api_txt_lines.dsc_txt_wrap")
        summary = summary_tag.get_text() if summary_tag else "요약 없음"

        # 언론사 이름
        press_tag = news.select_one("a.info.press")
        press_name = press_tag.get_text() if press_tag else "언론사 없음"

        # 기사 날짜
        date_tag = news.select("span.info")
        if date_tag:
            relative_date = date_tag[0].get_text()
            current_date = datetime.now().strftime("%Y.%m.%d")
            date = f"{current_date} ({relative_date})"
        else:
            date = "날짜 정보 없음"

        # 데이터 저장
        data.append({
            'title': title,
            'url': article_url,
            'summary': summary,
            'press': press_name,
            'date': date
        })

    # 만약 설정한 max_news 개수 이상이면 종료
    if len(data) >= max_news:
        break

# 결과 출력
for item in data:
    print(f"제목: {item['title']}")
    print(f"언론사: {item['press']}")
    print(f"날짜: {item['date']}")
    print(f"URL: {item['url']}")
    print(f"요약: {item['summary']}")
    print("-" * 400)


In [4]:
pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 40.3 MB/s eta 0:00:00


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument('--headless')  # 창 없이 실행
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

query = "OpenAI"
start_date = "2025.02.01"
end_date = "2025.03.31"
max_news = 20
news_per_page = 10
data = []

for start in range(1, max_news, news_per_page):
    url = f"https://search.naver.com/search.naver?where=news&query={query}&ds={start_date}&de={end_date}&sort=0&start={start}"
    driver.get(url)
    time.sleep(2)  # JS 렌더링 대기

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    news_list = soup.select("ul.list_news > li.bx")

    for news in news_list:
        title_tag = news.select_one("a.news_tit")
        if not title_tag:
            continue
        title = title_tag.get_text()
        article_url = title_tag['href']
        summary_tag = news.select_one("a.api_txt_lines.dsc_txt_wrap")
        summary = summary_tag.get_text() if summary_tag else "요약 없음"
        press_tag = news.select_one("a.info.press")
        press_name = press_tag.get_text() if press_tag else "언론사 없음"
        date_tag = news.select("span.info")
        date = date_tag[0].get_text() if date_tag else "날짜 정보 없음"

        data.append({
            'title': title,
            'url': article_url,
            'summary': summary,
            'press': press_name,
            'date': date
        })

    if len(data) >= max_news:
        break

driver.quit()

# 결과 출력
for item in data:
    print(f"제목: {item['title']}")
    print(f"언론사: {item['press']}")
    print(f"날짜: {item['date']}")
    print(f"URL: {item['url']}")
    print(f"요약: {item['summary']}")
    print("-" * 400)


ModuleNotFoundError: No module named 'selenium'

In [ ]:

import random

# 기사 목록 출력
print("기사 목록:")
for idx, article in enumerate(data):
    print(f"{idx + 1}: {article['title']}")

# 사용자 입력으로 기사 선택
if len(data) >= 2:
    try:
        selected_indices = input("=====선택할 기사 번호 두 개를 입력하세요 (예: 1 3): ").split()
        selected_indices = [int(i) - 1 for i in selected_indices]

        if len(selected_indices) != 2 or any(i < 0 or i >= len(data) for i in selected_indices):
            raise ValueError("잘못된 입력입니다. 올바른 기사 번호 두 개를 입력하세요.")

        selected_articles = [data[selected_indices[0]], data[selected_indices[1]]]
        print("---"*50)
        print("선택된 기사 제목:")
        print("1:", selected_articles[0]['title'])
        print("2:", selected_articles[1]['title'])
    except ValueError as e:
        print(f"입력 오류: {e}")
else:
    print("기사가 충분하지 않습니다.")

기사 목록:
기사가 충분하지 않습니다.


In [ ]:
import random

# 기사 2개 랜덤 선별
if len(data) >= 2:
    selected_articles = random.sample(data, 2)
    print("선택된 기사 제목:")
    print("1:", selected_articles[0]['title'])
    print("2:", selected_articles[1]['title'])
else:
    print("기사가 충분하지 않습니다.")


선택된 기사 제목:
1: 인공지능 기술주 '랠리' 올해도 지속 전망, AI 사업 '돈 버는 시대' 다가... 
2: 폴라리스오피스 주가 기세등등... 테슬라 "그록3 곧 테슬라 차량에 탑... 


In [ ]:
import os
from getpass import getpass

# 사용자에게 API 키를 입력받아 환경변수에 저장
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key: ··········


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

prompt = f"""
아래의 두 기사를 바탕으로 '제목, 서론, 본론, 결론'의 형식을 갖춘 하나의 자연스러운 뉴스 기사로 작성해주세요. 서론:, 본론:, 결론: 이라고 적지말고 그냥 한 개의 글로 적어주세요.
제목도 최적화 검색 엔진을 통해 사람들이 많이 클릭할 수 있도록 해줘.
각 구성 요소는 명확히 구분하고, 줄바꿈을 통해 가독성을 높여주세요.
또한 실제 기사처럼 글자수를 1000자~2000자 사이로 맞추고 내용을 풍부하게 적어주세요.
마지막으로, 이 기사를 읽고 느낀 것을 다양한 관점에서 적어주세요.

기사 1:
{selected_articles[0]['summary']}

기사 2:
{selected_articles[1]['summary']}

새로 작성된 기사:
"""


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=2500,
    temperature=0.7
)

revised_article = response.choices[0].message.content.strip()
print("[재편집된 기사]")
print(revised_article)


[재편집된 기사]
리버풀, 토너먼트 일정에서 PSG와 인터밀란과 맞붙어

리그 페이즈에서 워낙 좋은 성적을 거뒀기 때문에 수월한 토너먼트 일정을 기대했던 리버풀은 PSG와 인터밀란과의 맞대결을 맞이하게 되었다.

리그앙에는 PSG가 있다. PSG는 리그 22경기를 치르면서 단 1경기도 지지 않았다. 17승 5무로 승점 56점을 쌓아 강력한 팀력을 보여주고 있다. 이에 리버풀은 높은 경기력을 발휘해야 할 것으로 보인다.

16강에서는 이탈리아의 강호, 인터밀란과 맞붙게 되었다. 인터밀란은 챔피언스리그 결승전까지 진출한 경험이 있으며, 강력한 수비와 빠른 역습이 특징이다. 현재 세리에A에서도 1위를 달리고 있는 팀으로, 페예노르트가 어려움을 겪을 수밖에 없는 상황이다.

리버풀은 PSG와 인터밀란과의 경기에서 뛰어난 전략과 팀워크를 펼쳐야 할 것으로 보인다. 어려운 상황 속에서도 최선을 다해 전력을 다해야 할 시기가 도래했다.결론:

이 기사를 읽고 느낀 점:
리버풀은 성적이 좋았지만, 강적인 PSG와 인터밀란과의 경기에서는 높은 경기력을 요구받을 것으로 보인다. 팀의 전략과 팀워크가 중요해 보이는데, 어려운 상황에서도 최선을 다해야 할 필요성을 느꼈다. 특히 인터밀란은 강력한 수비와 빠른 역습이 특징이므로, 리버풀은 대비책을 세워야 할 것으로 보인다. 전략적인 접근이 필요한 상황에서 팀의 결속력이 시험을 받을 것으로 보여 조망이 필요하다.


In [ ]:
!pip install requests PyJWT

In [ ]:
!pip install PyJWT requests

In [ ]:
# main prompt
import os
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 프롬프트 설정
prompt = f"""
아래의 두 기사의 내용을 모두 담아서, '제목, 서론, 본론, 결론'의 형식을 갖춘 하나의 자연스러운 뉴스 기사를 작성해주세요.

아래 사항을 반드시 지켜주세요:

1. 반드시 기사의 맨 첫 줄에 [] 안에 매력적이고 클릭하고 싶어질 만한 제목을 작성해주세요.
2. 제목과 본문 내용을 검색 엔진 최적화(SEO)를 고려해서 작성해주세요.
3. 서론, 본론, 결론이라고 명시하지 말고, 자연스럽게 누군가와 대화하는 듯한 느낌으로 글을 작성해주세요. 모르는 사람에게 알려주듯이 친근하게 말하는 방법도 좋습니다.
4. 각 구성 요소는 줄바꿈을 통해 명확히 구분하여 가독성을 높여주세요.
5. 실제로 누군가와 대화하는 것처럼 글자 수를 1200자~1600자 사이로 맞춰 풍부한 내용을 담아주세요.
6. 대화를 통해 누군가에게 설명해주듯이 말하면서 중간중간 자문자답 형식 1~2번 포함되면 정말 좋을 거 같아요. (필수로 질문 문장에서는 꼭 줄바꿈을 해줘.)
7. 2개의 기사 내용이 골고루 들어가하고 기사의 내용에 기반하여 사실만 전달해주세요.

기사 1:
{selected_articles[0]['summary']}

기사 2:
{selected_articles[1]['summary']}

새로 작성된 기사:
"""

# GPT-4o를 사용하여 기사 생성
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=3000,
    temperature=0.7
)

# 기사 초안 저장
revised_article = response.choices[0].message.content.strip()

# Colab에서 직접 수정할 수 있는 입력창 생성
text_area = widgets.Textarea(
    value=revised_article,
    placeholder="여기에 기사를 수정하세요...",
    layout=widgets.Layout(width="100%", height="400px")
)

# "확정" 버튼 추가
confirm_button = widgets.Button(description="✅ 기사 확정")

# 확정된 기사를 저장할 변수
final_article = ""

# 버튼 클릭 이벤트 핸들러
def on_confirm_clicked(b):
    global final_article
    final_article = text_area.value
    print("\n✅ 최종 확정된 기사:\n")
    print(final_article)

confirm_button.on_click(on_confirm_clicked)

# UI 표시
display(text_area, confirm_button)


Textarea(value="[AI 혁신, 스마트폰과 교통 시스템에 불러온 변화]\n\n최근 IT 업계에서는 인공지능(AI)의 혁신이 눈에 띄게 진행되고 있어요. 특히, 삼성의 갤럭시 S 시리즈에 …

Button(description='✅ 기사 확정', style=ButtonStyle())


✅ 최종 확정된 기사:

[AI 혁신, 스마트폰과 교통 시스템에 불러온 변화]

최근 IT 업계에서는 인공지능(AI)의 혁신이 눈에 띄게 진행되고 있어요. 특히, 삼성의 갤럭시 S 시리즈에 오픈 AI의 챗GPT가 탑재될 가능성이 높아 주목받고 있죠. 이와 함께 애플도 인공지능 모델인 애플 인텔리전스와 음성 비서 시리(Siri)를 iOS 18.4 베타 버전에 통합하며 AI 기술을 향한 발걸음을 이어가고 있습니다. 그렇다면, AI가 우리 일상에 어떤 변화를 가져올까요?

먼저, 삼성 갤럭시 S 시리즈에 챗GPT가 탑재된다면 어떤 변화가 있을까요? 이 기술은 사용자와의 상호작용을 더욱 자연스럽게 만들어 줄 것으로 기대됩니다. 사용자가 질문을 던지면, 챗GPT는 그 질문에 대한 맥락을 이해하고 적절한 답변을 제공할 수 있게 될 것입니다. 예를 들어, '오늘 날씨 어때?'라고 물으면, 단순한 날씨 정보뿐만 아니라, 사용자의 일정을 고려한 추천까지 가능하게 될지도 모릅니다.

그런데 AI 응답의 신뢰도는 어떨까요? 최근 SRT와 관련하여 AI의 응답 신뢰도를 분석한 결과, 챗GPT를 포함한 국내외 AI 도구에 질문을 던졌을 때 20건 중 17건이나 잘못된 답변을 제공했다고 해요. AI가 아직 완벽하지 않다는 것을 보여주는 사례죠. 예를 들어, SRT 열차 내에서 컵라면을 먹기 위해 승무원에게 요청해야 한다는 잘못된 답변도 있었어요. 그럼에도 불구하고, AI 기술의 발전은 멈추지 않을 것입니다.

애플도 이 흐름에 따라 음성 비서 시리의 기능을 더욱 강화하고 있습니다. 시리가 애플 인텔리전스와 결합되면, 사용자 경험이 더욱 개인화될 것입니다. 이는 단순한 정보 전달을 넘어, 사용자의 일상 속에서 더 유용하게 활용될 수 있도록 돕는 것이죠.

결국, AI의 도입과 발전은 우리의 삶에 긍정적인 영향을 미칠 가능성이 큽니다. 물론, 현재로서는 AI가 완벽하지 않다는 점을 인식하고, 이를 보완해 나가야 할 필요가 있습니다. 앞으로 AI가 가져올 다양한 변화와 혁신을 기대하며, 

# 새 섹션

In [ ]:
 import os
import requests
from getpass import getpass
from openai import OpenAI

# 🔐 사용자에게 OpenAI API Key 입력받고 환경변수로 저장
os.environ['OPENAI_API_KEY'] = getpass('🔑 Enter your OpenAI API key: ')
openai_api_key = os.getenv("OPENAI_API_KEY")

# 👻 Ghost 관련 키는 입력으로 받음
ghost_admin_api_url = input("🌐 Ghost Admin API URL (예: https://yourblog.com/ghost/api/admin): ").strip()
ghost_admin_api_key = input("🧙 Ghost Admin API Key: ").strip()

# OpenAI 클라이언트 생성
client = OpenAI(api_key=openai_api_key)

# 1. GPT로 기사 + 지브리 이미지 프롬프트 생성
def generate_article_and_prompt(article1, article2):
    prompt = f"""
    아래 두 기사의 내용을 바탕으로, 자연스럽고 풍부한 뉴스 기사를 작성해주세요.
    또한 기사에 어울리는 지브리 스타일의 이미지 생성을 위한 프롬프트도 함께 제시해주세요.

    기사 1:
    {article1}

    기사 2:
    {article2}

    출력 예시:
    ---
    [지브리 스타일 뉴스 제목]
    기사 본문...

    이미지 프롬프트:
    A peaceful countryside scene at sunrise in Studio Ghibli style, warm tones, animated feel
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2500,
        temperature=0.7
    )
    return response.choices[0].message.content

# 2. DALL·E로 이미지 생성
def generate_image_url(image_prompt):
    image_response = client.images.generate(
        model="dall-e-3",
        prompt=image_prompt,
        size="1024x1024",
        n=1
    )
    return image_response.data[0].url

# 3. Ghost에 이미지 업로드
def upload_image_to_ghost(image_url):
    image_data = requests.get(image_url).content
    headers = {
        'Authorization': f'Ghost {ghost_admin_api_key}'
    }
    files = {'file': ('ghibli.jpg', image_data, 'image/jpeg')}
    response = requests.post(
        f"{ghost_admin_api_url}/images/upload/",
        headers=headers,
        files=files
    )
    response.raise_for_status()
    return response.json()['url']

# 4. Ghost에 포스트 업로드
def post_to_ghost(title, html_content, image_url):
    headers = {
        'Authorization': f'Ghost {ghost_admin_api_key}',
        'Content-Type': 'application/json'
    }
    post_data = {
        "posts": [{
            "title": title,
            "html": html_content,
            "feature_image": image_url,
            "status": "published"
        }]
    }
    response = requests.post(
        f"{ghost_admin_api_url}/posts/?source=html",
        headers=headers,
        json=post_data
    )
    response.raise_for_status()
    return response.json()

# 5. 실행 함수
def main():
    article1 = input("🗞️ 기사 1 요약을 입력하세요:\n")
    article2 = input("🗞️ 기사 2 요약을 입력하세요:\n")

    print("📰 GPT로 기사 생성 중...")
    full_output = generate_article_and_prompt(article1, article2)

    if "이미지 프롬프트:" in full_output:
        content, image_prompt = full_output.strip().split("이미지 프롬프트:")
    else:
        raise ValueError("이미지 프롬프트를 추출할 수 없습니다.")

    title_line = content.strip().split("\n")[0]
    title = title_line.strip("[]")

    print("🎨 이미지 생성 중...")
    image_url = generate_image_url(image_prompt.strip())

    print("⬆️ Ghost에 이미지 업로드 중...")
    ghost_image_url = upload_image_to_ghost(image_url)

    print("📝 포스트 업로드 중...")
    result = post_to_ghost(title, content.strip(), ghost_image_url)

    print("✅ 업로드 완료:", result["posts"][0]["url"])

if __name__ == "__main__":
    main()


In [ ]:
#Posting Code
import jwt
import requests
import datetime
import json

# 네 Ghost 블로그 정보
admin_api_key = '67aef8b28f249f00015b6167:581583e8c8c1ecda1edc60f35017f171ca640b46492a701e6e60f3f72445d277'  # 네 API 키
api_url = 'https://scrollaix.ghost.io/ghost/api/admin/posts/'  # 네 블로그 URL

# JWT 생성
id, secret = admin_api_key.split(':')
iat = int(datetime.datetime.now().timestamp())

header = {'alg': 'HS256', 'typ': 'JWT', 'kid': id}
payload = {
    'iat': iat,
    'exp': iat + 5 * 60,
    'aud': '/ghost/api/admin/'
}

token = jwt.encode(payload, bytes.fromhex(secret), algorithm='HS256', headers=header)

# 제목과 본문 분리
title, content = final_article.split('\n', 1)
title = title.replace('[제목]: ', '').strip()
content = content.strip()

# 포스팅 데이터
headers = {
    'Authorization': f'Ghost {token}',
    'Accept-Version': 'v5.109'
}

import json

mobiledoc_content = {
    "version": "0.3.1",
    "atoms": [],
    "cards": [],
    "markups": [],
    "sections": [[1, "p", [[0, [], 0, content]]]]  # ✅ 본문을 Mobiledoc 포맷으로 추가
}

post_data = {
    'posts': [{
        'title': title,
        'mobiledoc': json.dumps(mobiledoc_content),  # ✅ Mobiledoc 포맷 적용
        'status': 'published'
    }]
}


# Ghost에 포스팅 요청
response = requests.post(api_url, json=post_data, headers=headers)


# 결과 확인
if response.status_code == 201:
    post_url = response.json()['posts'][0]['url']
    print(f"🎉포스팅 완료: {post_url}")
else:
    print(f"🚨포스팅 실패: {response.status_code}, {response.text}")


🎉포스팅 완료: https://scrollaix.com/ai-hyeogsin-seumateupongwa-gyotong-siseuteme-bulreoon-byeonhwa-3/
